# 1.Data Collection


## 라이브러리 가져오기

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Robot, Camera, bgr8_to_jpeg

from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time
from SCSCtrl import TTLServo

robot = Robot()

## 카메라 각도 세팅하기  

In [ ]:
TTLServo.servoAngleCtrl(5, 50, 1, 100)
print('Camera Ready!')

## AGV 제어를 위한 컨트롤러 추가하기

In [3]:
# create buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)
# 레이아웃 생성 후,버튼 5개 생성

# display buttons
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])
display(controls_box)

## 컨트롤러에서 버튼 동작 시 호출하는 함수

In [4]:
def stop(change): # 정지
    robot.stop()
    
def step_forward(change): # 전진
    robot.forward(0.4)

def step_backward(change): # 후진
    robot.backward(0.4)

def step_left(change): # 좌회전
    robot.left(0.3)
    time.sleep(0.5)
    robot.stop()

def step_right(change): # 우회전
    robot.right(0.3)
    time.sleep(0.5)
    robot.stop()

stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

## 데이터 수집 경로 설정

In [ ]:
DATASET_DIR = 'dataset_xy_test'

try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

## 카메라 송출하기

In [6]:
camera = Camera()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)
display(widgets.HBox([image_widget, target_widget]))

count_widget = widgets.IntText(description='count',
 value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))
save_button = widgets.Button(description='Save', button_style='success')

display(widgets.VBox([x_slider, y_slider, count_widget, save_button]))

## 데이터 수집하기

In [7]:
def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

def save_snapshot():
    uuid = xy_uuid(x_slider.value, y_slider.value)
    image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_widget.value)
    count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))
    
save_button.on_click(lambda x: save_snapshot())

데이터 수집 절차는 다음과 같습니다.  
1. Camera 의 실시간 영상을 보고, AGV가 나아가야 하는 target의 위치에 녹색 점을 둔다.
2. save 버튼을 눌러서 저장한다.
3. 수집한 데이터 파일은 dataset_xy 폴더에 저장되며, 아래와 같은 이름 형식을 지닌다.
> \"xy_xValue_yValue_UUID.jpg\"

train_model 챕터에서 model 을 학습시킬 때는 이미지 파일을 불러와서 파일 이름에서 x,y 좌표 값을 파싱해서 사용한다.

## 프로젝트 종료하기

In [8]:
camera.unobserve_all()
camera.stop()
robot.stop()

## 데이터 압축해서 내보내기

In [ ]:
!zip -r -q road_following_{DATASET_DIR}.zip {DATASET_DIR}